# Lab 06: Reshaping

## #1
Load in the flights data that we've used before.

In [1]:
import pandas as pd

In [2]:
flights = pd.read_csv('../data/flights.csv')

## #2
Using a pivot table, find the median flight time to go from each origin to each destination.
Save that resulting pivot table in a variable called "pivoted_df".

We want to create a pivot table where the categories (which will become the rows/columns) are *origin* and *dest*.
The values are the *median* of *air_time*.

In [3]:
# You could reverse the columns= and index= arguments and get the same result, transposed.
pivoted_df = flights.pivot_table(columns='dest', index='origin', values='air_time', aggfunc='median')
pivoted_df

dest,ABQ,ACK,ALB,ANC,ATL,AUS,AVL,BDL,BGR,BHM,...,SNA,SRQ,STL,STT,SYR,TPA,TUL,TVC,TYS,XNA
origin,,,,,,,,,,,,,,,,,,,,,
EWR,NaN,NaN,31.0,414.5,111.0,210.0,89.0,25.0,NaN,NaN,...,329.0,NaN,133.0,205.0,39.0,142.0,176.0,95.0,96.0,165.0
JFK,246.0,41.0,NaN,NaN,111.0,212.0,NaN,NaN,NaN,117.0,...,NaN,151.0,129.0,202.0,44.0,143.0,NaN,NaN,NaN,NaN
LGA,NaN,NaN,NaN,NaN,112.0,NaN,91.5,NaN,54.0,121.0,...,NaN,150.0,132.0,NaN,38.0,144.0,NaN,93.0,96.0,171.0


## #3
Experiment with the `.dropna` method -- consult its documentation using the `?`.
Use it to drop destinations in your result from #2 where at least one value is `NaN`.
Store this "cleaned" DataFrame in a variable called "clean_df".
- destinations may be either rows or columns depending on how you did #2

In [4]:
# Setting axis=1 drops columns with NaNs, and how=any means that we should
# drop any columns where *at least one* value is NaN. how=any is th default
# but I like to be explicit.
clean_df = pivoted_df.dropna(axis=1, how='any')
# If you set destinations to be rows in #2, you would use this line:
# clean_df = pivoted_df.dropna(axis=0, how='any')
clean_df

dest,ATL,BNA,BOS,BTV,BUF,BWI,CHS,CLE,CLT,CMH,...,PIT,PWM,RDU,RIC,ROC,RSW,SDF,STL,SYR,TPA
origin,,,,,,,,,,,,,,,,,,,,,
EWR,111.0,113.0,40.0,46.0,51.0,37.0,94.0,66.0,83.0,75.0,...,56.0,48.0,69.0,53.0,45.0,155.0,105.0,133.0,39.0,142.0
JFK,111.0,116.0,38.0,47.0,57.0,38.0,94.0,76.5,86.0,76.0,...,66.0,48.0,71.0,56.0,51.0,159.0,103.0,129.0,44.0,143.0
LGA,112.0,112.0,37.0,42.0,50.0,40.0,91.0,69.0,85.0,77.0,...,57.0,44.0,70.0,54.0,45.0,159.0,105.0,132.0,38.0,144.0


## #4
Using your result from #3, extract the destination airport codes as a list.
What is the context of these airport codes?
i.e. When might it be helpful to have prepared a list of these airport codes?

In [5]:
# If destinations are your columns, this is probably the simplest approach:
result_dest = list(clean_df.columns)
# If destinations were rows in your case, this would work:
# result_dest = list(clean_df.index)

# Print just the first 5 values since there are so many
result_dest[:5]

['ATL', 'BNA', 'BOS', 'BTV', 'BUF']

The remaining destinations in this data are destinations that received flights from *all* of the origins in the dataset -- NaNs in the median flight time likely represent that there are no flights that made that journey.

## #5
Reload the planes data that we've used before.

In [6]:
planes = pd.read_csv('../data/planes.csv')

## #6
Melt this data, using all columns except tailnum as your value variables.
Use tailnum as your identifier.
Store the result as "melted_df".

In [7]:
melted_df = planes.melt(id_vars='tailnum', value_vars=['year', 'type', 'manufacturer', 'model', 'engines', 'seats', 'speed', 'engine'])
# If we don't set value_vars specifically, the default is all columns other than our id_vars --
# so in this case there is a simpler way to achieve the same result:
# melted = planes.melt(id_vars='tailnum')
melted_df.head()

,tailnum,variable,value
0,N10156,year,2004
1,N102UW,year,1998
2,N103US,year,1999
3,N104UW,year,1999
4,N10575,year,2002


## #7
Use the .pivot method to "widen" the data from #6 and put it back in the form it started in.

In [8]:
unmelted_df = melted_df.pivot(columns='variable', index='tailnum', values='value')
unmelted_df.head()

variable,engine,engines,manufacturer,model,seats,speed,type,year
tailnum,,,,,,,,
N10156,Turbo-fan,2,EMBRAER,EMB-145XR,55,NaN,Fixed wing multi engine,2004
N102UW,Turbo-fan,2,AIRBUS INDUSTRIE,A320-214,182,NaN,Fixed wing multi engine,1998
N103US,Turbo-fan,2,AIRBUS INDUSTRIE,A320-214,182,NaN,Fixed wing multi engine,1999
N104UW,Turbo-fan,2,AIRBUS INDUSTRIE,A320-214,182,NaN,Fixed wing multi engine,1999
N10575,Turbo-fan,2,EMBRAER,EMB-145LR,55,NaN,Fixed wing multi engine,2002


This isn't *exactly* like the original data.
For example, 'tailnum' has become the index -- but originally we had a range index, simple increasing integers.
We can fix that.

In [9]:
unmelted_df = unmelted_df.reset_index()
unmelted_df.head()

variable,tailnum,engine,engines,manufacturer,model,seats,speed,type,year
0,N10156,Turbo-fan,2,EMBRAER,EMB-145XR,55,NaN,Fixed wing multi engine,2004
1,N102UW,Turbo-fan,2,AIRBUS INDUSTRIE,A320-214,182,NaN,Fixed wing multi engine,1998
2,N103US,Turbo-fan,2,AIRBUS INDUSTRIE,A320-214,182,NaN,Fixed wing multi engine,1999
3,N104UW,Turbo-fan,2,AIRBUS INDUSTRIE,A320-214,182,NaN,Fixed wing multi engine,1999
4,N10575,Turbo-fan,2,EMBRAER,EMB-145LR,55,NaN,Fixed wing multi engine,2002


#### Challenge Questions
1. Using your result from #3 again, run the following code:
    - If your row labels (index) are **origin** airport: `clean_df.reset_index().melt(id_vars='origin', value_name='air_time').dropna()`
    - If your row labels are **dest**ination airport: `clean_df.reset_index().melt(id_vars='dest', value_name='air_time').dropna()`

2. Can you replicate the values in this resulting DataFrame using the original flights data and and performing a groupby and aggregate?
    - *Hint: passing `as_index=False` to `.groupby` will keep pandas from moving your grouping columns into the index, making your result look more like the data from #1.*

3. Do you have any intuition as to why this works?

In [17]:
clean_df.reset_index().melt(id_vars='origin', value_name='air_time').dropna()

,origin,dest,air_time
0,EWR,ATL,111.0
1,JFK,ATL,111.0
2,LGA,ATL,112.0
3,EWR,BNA,113.0
4,JFK,BNA,116.0
...,...,...,...
121,JFK,SYR,44.0
122,LGA,SYR,38.0
123,EWR,TPA,142.0
124,JFK,TPA,143.0


The line to do this with a groupby isn't too bad.

The resulting data won't be *quite* the same because we dropped all destinations that didn't have a flight from all 3 origins –- so `same_df`, below, will have rows that aren't in `clean_df`.

In [19]:
same_df = flights.groupby(['origin', 'dest'], as_index=False).agg({'air_time': 'median'})
same_df

,origin,dest,air_time
0,EWR,ALB,31.0
1,EWR,ANC,414.5
2,EWR,ATL,111.0
3,EWR,AUS,210.0
4,EWR,AVL,89.0
...,...,...,...
219,LGA,SYR,38.0
220,LGA,TPA,144.0
221,LGA,TVC,93.0
222,LGA,TYS,96.0


Why does this work?
As we discussed in class, a pivot table is fundamentally a two-level group-by.